In [ ]:
def extractSeqPositionalInfo(gbk_file: str) -> str:
    """
    Extract contig and locus position from gbk for given sequence
    """
    return None

In [1]:
from Bio import SeqIO, SeqRecord


file = "/home/robaina/Documents/Pynteny/MG1655.gb"
gbk_contigs = list(SeqIO.parse(file, 'genbank'))

In [1]:
from pynteny.preprocessing import assignGeneLocationToRecords



gbk_file = "/home/robaina/Documents/Pynteny/MG1655.gb"

assignGeneLocationToRecords(
    gbk_file=gbk_file,
    output_fasta=None,
    nucleotide=False
)

In [ ]:
from pynteny.wrappers import runProdigal

# Run prodigal on assembly to obtain gene translation and positional info

runProdigal(
    input_file="genome_assembly.fasta",
    output_prefix=None,
    output_dir=None
)

In [2]:
# In case of split genomes, each part comes in a different gbk file,this would happen with differnt contigs as well.
# So, basically, I only need the gbk file, which can be obtanined from prokka with the genome assembly in fasta format.
# Genes appear to be sorted by loci in gbk files
# Actually, the departing point for a hmmer search with synteny info would be a fasta file containing reference sequences...
# so, we would need to obtain these sequences from the gbk file or map them somehow?
# Perhaps better to store positional info in a dict, this way no need to alter contig/sequence names

from Bio import SeqIO, SeqRecord


file = "/home/robaina/Documents/Pynteny/MG1655.gb"


gbk_contigs = list(SeqIO.parse(gbk_file, 'genbank'))

with open("LongLabels.fasta", "w") as outfile:

    for gbk_contig in gbk_contigs:
        
        gene_counter = 0
        for feature in gbk_contig.features:

            if "cds" in feature.type.lower():
                
                name = feature.qualifiers["locus_tag"][0].replace('_', '.')
                start, end, strand = str(feature.location.start), str(feature.location.end), feature.location.strand
                start = start.replace(">", "").replace("<", "")
                end = end.replace(">", "").replace("<", "")
                strand_sense = "neg" if strand == -1 else "pos"

                header = f">{name}__{gbk_contig.name.replace('_', '')}_{gene_counter}_{start}_{end}_{strand_sense}\n"
                if "translation" in feature.qualifiers:
                    translation = feature.qualifiers["translation"][0]

                    header = f">{name}__{gbk_contig.name.replace('_', '')}_{gene_counter}_{start}_{end}_{strand_sense}\n"
                    outfile.write(header)
                    outfile.write(translation + "\n")

                    gene_counter += 1